In [18]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [19]:
readIn = pd.read_csv("df_prep.csv")
df = pd.DataFrame(readIn)


In [20]:
readIn = pd.read_csv("data_submission_example.csv")
df_target = pd.DataFrame(readIn)
# df_target.head(20)

# print(df_target.shape)
# print(df_target['BORROWER_ID'].unique)

In [21]:
# print(df.shape)

# index_to_keep = df['BORROWER_ID'].isin(df_target['BORROWER_ID'])

# df = df[index_to_keep]

# print(df.shape)

In [22]:
df.columns

Index(['BORROWER_ID', 'CONTRACT_DATE_OF_LOAN_AGREEMENT', 'CONTRACT_INCOME',
       'CONTRACT_INSTALMENT_AMOUNT', 'CONTRACT_INTEREST_RATE', 'CONTRACT_LGD',
       'CONTRACT_LOAN_AMOUNT', 'CONTRACT_LOAN_TO_VALUE_RATIO',
       'CONTRACT_MARKET_VALUE', 'CONTRACT_MATURITY_DATE',
       ...
       'CONTRACT_REFINANCED=_10001.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_0.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_100001.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_100002.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_100003.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_100004.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_110001.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_140001.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_140002.0',
       'CONTRACT_TYPE_OF_INTEREST_REPAYMENT=_140003.0'],
      dtype='object', length=459)

In [12]:
cols= ['CONTRACT_DATE_OF_LOAN_AGREEMENT','CONTRACT_MATURITY_DATE', 'TARGET_EVENT_DAY' ]

In [17]:
df2=df[cols]


df3=df2[df2['TARGET_EVENT_DAY'].notnull()]

print(len(df3))


print(len(df3[df3['TARGET_EVENT_DAY']>df3['CONTRACT_MATURITY_DATE']]))

51380
2358


In [ ]:
def datagen_modeA(origdf,startnap, endnap):
    df2 = origdf[ origdf['CONTRACT_DATE_OF_LOAN_AGREEMENT']>=startnap].copy()
    df = df2[ df2['CONTRACT_DATE_OF_LOAN_AGREEMENT']<endnap].copy()
    
    cust_df = pd.DataFrame({'BORROWER_ID':df['BORROWER_ID'].unique()})
    # TARGET VALTOZO KESZITES
    feltetel=(df['TARGET_EVENT_DAY']>endnap) & (df['TARGET_EVENT_DAY']<endnap+2*365) & (df['TARGET_EVENT']=='K') 
    df['target']=0
    df.loc[feltetel,'target']=1
    
    stat = df.groupby("BORROWER_ID",as_index=False).agg({'target':'max'})
    cust_df=cust_df.merge(stat,on='BORROWER_ID',how='left')
    cust_df['target']=cust_df['target'].fillna(0)  
    
    # BEMENO_VALTOZOKAT
    stat = df.groupby("BORROWER_ID",as_index=False).agg(
        {'CONTRACT_ID':'count','CONTRACT_LOAN_AMOUNT':'sum'})
    stat.columns=['BORROWER_ID','in_contract_num','in_loan_amount_sum']
    cust_df=cust_df.merge(stat,on='BORROWER_ID',how='left')
    return cust_df

R1 = datagen_modeA(df,0,365)
R2 = datagen_modeA(df,365*2,365*3)

# SAVING RESULTS

In [54]:
df=df.drop_duplicates()

print(df.shape)
df.to_csv('results.csv', index=False)

(1505896, 458)


In [55]:
len(df)

1505896